In [52]:
import numpy as np
import pandas as pd
import seaborn as sns
    

In [53]:
df=sns.load_dataset('titanic')
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


In [54]:
df2 = df.iloc[:, :8]
df2

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [55]:
df2['deck'] = df.deck

In [56]:
# NaN 처리
df2.isna().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
deck        688
dtype: int64

In [57]:
# 나이는 평균으로 대체
df2.age.fillna(df.age.mean(), inplace=True)


In [58]:
df2.isna().sum()

survived      0
pclass        0
sex           0
age           0
sibsp         0
parch         0
fare          0
embarked      2
deck        688
dtype: int64

In [59]:
# deck 열 삭제
df2.drop(columns='deck', inplace=True)

# embark_town은 최빈값으로 대체

In [60]:
# embarked 의 최빈값 도출
df2.embarked.value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [61]:
df2.embarked.fillna('S', inplace=True)

In [62]:
df2

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,male,22.000000,1,0,7.2500,S
1,1,1,female,38.000000,1,0,71.2833,C
2,1,3,female,26.000000,0,0,7.9250,S
3,1,1,female,35.000000,1,0,53.1000,S
4,0,3,male,35.000000,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S
887,1,1,female,19.000000,0,0,30.0000,S
888,0,3,female,29.699118,1,2,23.4500,S
889,1,1,male,26.000000,0,0,30.0000,C


In [64]:
# sex, embarked를 숫자로 변환
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df2['sex'] = encoder.fit_transform(df2.sex)
df2['embarked'] = encoder.fit_transform(df2.embarked)
df2.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2


- train/test 데이터셋으로 분리

In [66]:
df2.survived.value_counts()

0    549
1    342
Name: survived, dtype: int64

In [68]:
X=df2.iloc[:,1:].values
y=df2.survived.values
X.shape, y.shape

((891, 7), (891,))

In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X,y,stratify=y, test_size=0.2, random_state=2121
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 7), (179, 7), (712,), (179,))

- randomforest 모델 생성 및 학습

In [71]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2021)
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2021,
 'verbose': 0,
 'warm_start': False}

In [73]:
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=2021)

- 성능평가

In [75]:
rfc.score(X_test, y_test)

0.8603351955307262

- GridsearchCV수행

In [81]:
params = {
    'max_depth' : [3,4,5,6,7],
    'min_samples_split' : [2,3,4,5,6]

}

In [82]:
from sklearn.model_selection import GridSearchCV
grid_rf = GridSearchCV(rfc, param_grid=params, cv=3, scoring='accuracy')
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=2021),
             param_grid={'max_depth': [3, 4, 5, 6, 7],
                         'min_samples_split': [2, 3, 4, 5, 6]},
             scoring='accuracy')

In [83]:
grid_rf.best_params_

{'max_depth': 6, 'min_samples_split': 4}

In [84]:
grid_rf.best_score_

0.8118167098062855

In [85]:
grid_rf.score(X_test, y_test)

0.8659217877094972

In [87]:
best_rf=grid_rf.best_estimator_

In [88]:
best_rf.score(X_test, y_test)

0.8659217877094972

- 테스트 데이터에 적용

In [117]:
y_test[100], X_test[100]

(0,
 array([ 2.        ,  1.        , 29.69911765,  0.        ,  0.        ,
         0.        ,  2.        ]))

In [120]:
best_rf.predict(X_test[100].reshape(1,-1))

array([0], dtype=int64)

- 타이타닉 엉터리 분류기

In [115]:
from sklearn.base import BaseEstimator
class MyClassfier(BaseEstimator):
    # fit(), predict() method 는 재정의(overriding)
    def fit(self, X, y):
        pass
    def predict(self, X):
        pred = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            if X[i,1] ==0: #여성이면
                pred[i] = 1
        return pred


In [116]:
my_clf = MyClassfier()
my_clf.fit(X_train, y_train)
pred = my_clf.predict(X_test)